In [1]:
# import standard libraries for linear algebra, handling data and plotting
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('precision', 2)
pd.set_option('display.max_columns', 100)
#pd.options.display.max_columns = 100

sns.set(style="white", color_codes=True)

%matplotlib inline

In [2]:
# read the data into a Pandas DataFrame
train_data = pd.read_csv('house-prices/train.csv')
test_data = pd.read_csv('house-prices/test.csv')

combine = [train_data, test_data]
data = train_data

# show some samples
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
import datetime
import numpy as np
import pandas as pd
#import xgboost as xgb
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.grid_search import GridSearchCV
#from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from scipy.stats import skew

/opt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
def mean_squared_error_(ground_truth, predictions):
    return mean_squared_error(ground_truth, predictions) ** 0.5

RMSE = make_scorer(mean_squared_error_, greater_is_better=False)

In [5]:
outlier_idx = [4, 11, 13, 20, 46, 66, 70, 167, 178, 185, 199, 224, 261, 309, 313, 318, 349, 412, 423, 440, 454, 477, 478, 523, 540, 581, 588, 595, 654, 688, 691, 774, 798, 875, 898, 926, 970, 987, 1027, 1109, 1169, 1182, 1239, 1256, 1298, 1324, 1353, 1359, 1405, 1442, 1447]
print("%d outliers to be dropped" % len(outlier_idx))

print("before:", train_data.shape)
train_data.drop(train_data.index[outlier_idx], inplace=True)
print("after:", train_data.shape)

51 outliers to be dropped
before: (1460, 81)
after: (1409, 81)


In [6]:
all_data = pd.concat((train_data.loc[:,'MSSubClass':'SaleCondition'],
                      test_data.loc[:,'MSSubClass':'SaleCondition']))
print(all_data.shape)

(2868, 79)


In [7]:
print("before:", len(all_data.columns))
to_delete = ['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
all_data.drop(to_delete, axis=1, inplace=True)
print("after:", len(all_data.columns))

before: 79
after: 74


In [8]:
train_data["SalePrice"].head()

0    208500
1    181500
2    223500
3    140000
5    143000
Name: SalePrice, dtype: int64

In [9]:
train_data["SalePrice"] = np.log1p(train_data["SalePrice"])
train_data["SalePrice"].head()

0    12.25
1    12.11
2    12.32
3    11.85
5    11.87
Name: SalePrice, dtype: float64

In [10]:
# log transform skewed numeric features

numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index
print("Numeric features:", numeric_feats)

skewed_feats = train_data[numeric_feats].apply(lambda x: skew(x.dropna())) # compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
print("Skewed features:", skewed_feats)

Numeric features: Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')
Skewed features: Index(['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF2',
       'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtHalfBath', 'KitchenAbvGr', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal'],
      dtype='object')


In [11]:
all_data[skewed_feats].head()

,MSSubClass,LotFrontage,LotArea,MasVnrArea,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtHalfBath,KitchenAbvGr,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal
0,60,65.0,8450,196.0,0.0,150.0,856,854,0,1710,0.0,1,0,61,0,0,0,0,0
1,20,80.0,9600,0.0,0.0,284.0,1262,0,0,1262,1.0,1,298,0,0,0,0,0,0
2,60,68.0,11250,162.0,0.0,434.0,920,866,0,1786,0.0,1,0,42,0,0,0,0,0
3,70,60.0,9550,0.0,0.0,540.0,961,756,0,1717,0.0,1,0,35,272,0,0,0,0
5,50,85.0,14115,0.0,0.0,64.0,796,566,0,1362,0.0,1,40,30,0,320,0,0,700


In [12]:
all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
all_data[skewed_feats].head()

,MSSubClass,LotFrontage,LotArea,MasVnrArea,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtHalfBath,KitchenAbvGr,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal
0,4.11,4.19,9.04,5.28,0.0,5.02,6.75,6.75,0.0,7.44,0.00,0.69,0.00,4.13,0.00,0.00,0.0,0.0,0.00
1,3.04,4.39,9.17,0.00,0.0,5.65,7.14,0.00,0.0,7.14,0.69,0.69,5.70,0.00,0.00,0.00,0.0,0.0,0.00
2,4.11,4.23,9.33,5.09,0.0,6.08,6.83,6.77,0.0,7.49,0.00,0.69,0.00,3.76,0.00,0.00,0.0,0.0,0.00
3,4.26,4.11,9.16,0.00,0.0,6.29,6.87,6.63,0.0,7.45,0.00,0.69,0.00,3.58,5.61,0.00,0.0,0.0,0.00
5,3.93,4.45,9.56,0.00,0.0,4.17,6.68,6.34,0.0,7.22,0.00,0.69,3.71,3.43,0.00,5.77,0.0,0.0,6.55


In [13]:
all_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,4.11,RL,4.19,9.04,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.28,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,5.02,856.0,GasA,Ex,Y,SBrkr,6.75,6.75,0.0,7.44,1.0,0.00,2,1,3,0.69,Gd,8,Typ,0,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0.00,4.13,0.00,0.00,0.0,0.0,0.00,2,2008,WD,Normal
1,3.04,RL,4.39,9.17,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.00,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,5.65,1262.0,GasA,Ex,Y,SBrkr,7.14,0.00,0.0,7.14,0.0,0.69,2,0,3,0.69,TA,6,Typ,1,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,5.70,0.00,0.00,0.00,0.0,0.0,0.00,5,2007,WD,Normal
2,4.11,RL,4.23,9.33,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,5.09,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,6.08,920.0,GasA,Ex,Y,SBrkr,6.83,6.77,0.0,7.49,1.0,0.00,2,1,3,0.69,Gd,6,Typ,1,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0.00,3.76,0.00,0.00,0.0,0.0,0.00,9,2008,WD,Normal
3,4.26,RL,4.11,9.16,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.00,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,6.29,756.0,GasA,Gd,Y,SBrkr,6.87,6.63,0.0,7.45,1.0,0.00,1,0,3,0.69,Gd,7,Typ,1,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0.00,3.58,5.61,0.00,0.0,0.0,0.00,2,2006,WD,Abnorml
5,3.93,RL,4.45,9.56,Pave,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.00,TA,TA,Wood,Gd,TA,No,GLQ,732.0,Unf,0.0,4.17,796.0,GasA,Ex,Y,SBrkr,6.68,6.34,0.0,7.22,1.0,0.00,1,1,1,0.69,TA,5,Typ,0,Attchd,1993.0,Unf,2.0,480.0,TA,TA,Y,3.71,3.43,0.00,5.77,0.0,0.0,6.55,10,2009,WD,Normal


In [14]:
all_data = pd.get_dummies(all_data)
all_data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,...,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.11,4.19,9.04,7,5,2003,2003,5.28,706.0,0.0,5.02,856.0,6.75,6.75,0.0,7.44,1.0,0.00,2,1,3,0.69,8,0,2003.0,2.0,548.0,0.00,4.13,0.00,0.00,0.0,0.0,0.00,2,2008,0,0,0,1,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,3.04,4.39,9.17,6,8,1976,1976,0.00,978.0,0.0,5.65,1262.0,7.14,0.00,0.0,7.14,0.0,0.69,2,0,3,0.69,6,1,1976.0,2.0,460.0,5.70,0.00,0.00,0.00,0.0,0.0,0.00,5,2007,0,0,0,1,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,4.11,4.23,9.33,7,5,2001,2002,5.09,486.0,0.0,6.08,920.0,6.83,6.77,0.0,7.49,1.0,0.00,2,1,3,0.69,6,1,2001.0,2.0,608.0,0.00,3.76,0.00,0.00,0.0,0.0,0.00,9,2008,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,4.26,4.11,9.16,7,5,1915,1970,0.00,216.0,0.0,6.29,756.0,6.87,6.63,0.0,7.45,1.0,0.00,1,0,3,0.69,7,1,1998.0,3.0,642.0,0.00,3.58,5.61,0.00,0.0,0.0,0.00,2,2006,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
5,3.93,4.45,9.56,5,5,1993,1995,0.00,732.0,0.0,4.17,796.0,6.68,6.34,0.0,7.22,1.0,0.00,1,1,1,0.69,5,0,1993.0,2.0,480.0,3.71,3.43,0.00,5.77,0.0,0.0,6.55,10,2009,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [15]:
all_data = all_data.fillna(all_data.mean())
all_data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,...,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.11,4.19,9.04,7,5,2003,2003,5.28,706.0,0.0,5.02,856.0,6.75,6.75,0.0,7.44,1.0,0.00,2,1,3,0.69,8,0,2003.0,2.0,548.0,0.00,4.13,0.00,0.00,0.0,0.0,0.00,2,2008,0,0,0,1,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,3.04,4.39,9.17,6,8,1976,1976,0.00,978.0,0.0,5.65,1262.0,7.14,0.00,0.0,7.14,0.0,0.69,2,0,3,0.69,6,1,1976.0,2.0,460.0,5.70,0.00,0.00,0.00,0.0,0.0,0.00,5,2007,0,0,0,1,0,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,4.11,4.23,9.33,7,5,2001,2002,5.09,486.0,0.0,6.08,920.0,6.83,6.77,0.0,7.49,1.0,0.00,2,1,3,0.69,6,1,2001.0,2.0,608.0,0.00,3.76,0.00,0.00,0.0,0.0,0.00,9,2008,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,4.26,4.11,9.16,7,5,1915,1970,0.00,216.0,0.0,6.29,756.0,6.87,6.63,0.0,7.45,1.0,0.00,1,0,3,0.69,7,1,1998.0,3.0,642.0,0.00,3.58,5.61,0.00,0.0,0.0,0.00,2,2006,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
5,3.93,4.45,9.56,5,5,1993,1995,0.00,732.0,0.0,4.17,796.0,6.68,6.34,0.0,7.22,1.0,0.00,1,1,1,0.69,5,0,1993.0,2.0,480.0,3.71,3.43,0.00,5.77,0.0,0.0,6.55,10,2009,0,0,0,1,0,0,1,1,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [16]:
X_train = all_data[:train_data.shape[0]]
y_train = train_data.SalePrice
X_test = all_data[train_data.shape[0]:]

print(type(X_train), type(X_test), type(y_train))
print(X_train.shape, X_test.shape, y_train.shape)

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
(1409, 268) (1459, 268) (1409,)


In [17]:
def create_submission(algorithm, prediction, score):
    now = datetime.datetime.now()
    sub_file = 'house-prices/submission_' + algorithm + '_' + \
        str(round(score, 4)) + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    print('Creating submission: ', sub_file)
    pd.DataFrame({'Id': test_data['Id'].values, 'SalePrice': prediction}).to_csv(sub_file, index=False)

In [22]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [23]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [24]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, \
    param_distributions = random_grid, n_iter = 10, cv = 3, \
    verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train, y_train)

# view the best parameters from fitting the random search
pprint(rf_random.best_params_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True 
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, total=   3.8s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, total=   4.0s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=90, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=50, bootstrap=True, total=   4.4s
[CV] n_estimators=600, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=9

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 27.6min finished


{'bootstrap': True,
 'max_depth': 50,
 'max_features': 'auto',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 2000}


In [27]:
rfr = RandomForestRegressor(n_jobs=1, random_state=0)
param_grid = {}#'n_estimators': [500], 'max_features': [10,15,20,25], 'max_depth':[3,5,7,9,11]}

param_grid = {
    'bootstrap': [True],
    'max_depth': [50, 100],
    'max_features': ['auto'],
    'min_samples_leaf': [2],
    'min_samples_split': [5],
    'n_estimators': [200, 2000]}

#{'bootstrap': True, 'max_depth': 50, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 2000}

model = GridSearchCV(estimator=rfr, param_grid=param_grid, n_jobs=2, cv=10, scoring=RMSE, verbose=2)
model.fit(X_train, y_train)
score = -model.best_score_

print('Random forecast regression...')
print(rfr)
print('Best Params:\n', model.best_params_)
print('Best CV Score:', score)

y_pred = model.predict(X_test)

create_submission('rfr', np.expm1(y_pred), score)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200 
[CV] bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200 
[CV]  bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200 -  28.4s
[CV] bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200 
[CV]  bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200 -  29.2s
[CV] bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200 
[CV]  bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200 -  32.1s
[CV] bootstrap=True, max_depth=50, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200 
[CV]  bo

[CV] bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 
[CV]  bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 - 5.9min
[CV] bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 
[CV]  bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 - 5.9min
[CV] bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 
[CV]  bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 - 5.6min
[CV] bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 


[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 55.2min


[CV]  bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 - 5.7min
[CV] bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 
[CV]  bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 - 5.6min
[CV]  bootstrap=True, max_depth=100, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=2000 - 5.5min


[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed: 61.0min finished


Random forecast regression...
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)
Best Params:
 {'bootstrap': True, 'max_depth': 50, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 2000}
Best CV Score: 0.13025615895755732
Creating submission:  house-prices/submission_rfr_0.1303_2019-05-11-23-18.csv


In [32]:
gbr = GradientBoostingRegressor(random_state=0)
param_grid = {
    'n_estimators': [100, 250, 500, 750, 1000],
    'max_features': [5, 10, 15, 20],
    'max_depth': [4, 6, 8, 10, 12, 14],
    'learning_rate': [0.05, 0.1, 0.15, 0.2],
    'subsample': [0.4, 0.6, 0.8]
}

#Best Params: {'learning_rate': 0.05, 'max_depth': 4, 'max_features': 15, 'n_estimators': 1000, 'subsample': 0.6}
#{'max_depth': 6, 'max_features': 15, 'n_estimators': 500}

model = GridSearchCV(estimator=gbr, param_grid=param_grid, n_jobs=2, cv=10, scoring=RMSE, verbose=1)
model.fit(X_train, y_train)
score = -model.best_score_

print('Gradient boosted tree regression...')
print(gbr)
print('Best Params:\n', model.best_params_)
print('Best CV Score:', score)

y_pred = model.predict(X_test)

create_submission('gbr', np.expm1(y_pred), score)

Fitting 10 folds for each of 1440 candidates, totalling 14400 fits


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:   22.6s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:  4.3min
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed: 15.3min
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed: 29.2min
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed: 58.8min
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed: 111.4min
[Parallel(n_jobs=2)]: Done 2446 tasks      | elapsed: 180.4min
[Parallel(n_jobs=2)]: Done 3196 tasks      | elapsed: 266.4min
[Parallel(n_jobs=2)]: Done 4046 tasks      | elapsed: 340.3min
[Parallel(n_jobs=2)]: Done 4996 tasks      | elapsed: 390.5min
[Parallel(n_jobs=2)]: Done 6046 tasks      | elapsed: 471.4min
[Parallel(n_jobs=2)]: Done 7196 tasks      | elapsed: 563.8min
[Parallel(n_jobs=2)]: Done 8446 tasks      | elapsed: 617.8min
[Parallel(n_jobs=2)]: Done 9796 tasks      | elapsed: 698.9min
[Parallel(n_jobs=2)]: Done 11246 tasks      | elapsed: 778.2min
[Parallel(n_jobs=2)]: Done 12796 tasks      | elapsed: 857.4min

Gradient boosted tree regression...
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False)
Best Params:
 {'learning_rate': 0.05, 'max_depth': 4, 'max_features': 15, 'n_estimators': 1000, 'subsample': 0.6}
Best CV Score: 0.10358417355407268
Creating submission:  house-prices/submission_gbr_0.1036_2019-05-12-15-53.csv


In [ ]:
'''
xgbr = xgb.XGBRegressor(seed=0)
param_grid = {
#        'n_estimators': [500],
#        'learning_rate': [ 0.05],
#        'max_depth': [ 7, 9, 11],
#        'subsample': [ 0.8],
#        'colsample_bytree': [0.75,0.8,0.85],
}

model = GridSearchCV(estimator=xgbr, param_grid=param_grid, n_jobs=1, cv=10, scoring=RMSE)
model.fit(X_train, y_train)
score = -model.best_score_

print('eXtreme Gradient Boosting regression...')
print(xgbr)
print('Best Params:\n', model.best_params_)
print('Best CV Score:', score)

y_pred = model.predict(X_test)

create_submission('xgbr', np.expm1(y_pred), score)
'''

In [21]:
etr = ExtraTreesRegressor(n_jobs=1, random_state=0)
param_grid = {}#'n_estimators': [500], 'max_features': [10,15,20]}

model = GridSearchCV(estimator=etr, param_grid=param_grid, n_jobs=1, cv=10, scoring=RMSE)
model.fit(X_train, y_train)
score = -model.best_score_

print('Extra trees regression...')
print(etr)
print('Best Params:\n', model.best_params_)
print('Best CV Score:', score)

y_pred = model.predict(X_test)

create_submission('etr', np.expm1(y_pred), score)

Extra trees regression...
ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=1, min_samples_split=2,
          min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
          oob_score=False, random_state=0, verbose=0, warm_start=False)
Best Params:
 {}
Best CV Score: 0.1425835846562044
Creating submission:  house-prices/submission_etr_0.1426_2019-05-11-21-19.csv
